<a href="https://colab.research.google.com/github/bermanlabemory/gait_signatures/blob/main/Gait_Signatures_Script_3_Generate_Gait_Signatures.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## This script accesses the extracted Hs and Cs from 'Gait Signatures Script 2: Extract Internal Parameters.ipynb' and generates phase averaged signatures. 

**notes:** 
1. Externally and Self-driven signatures are generated


**Created by**: Taniel Winner

**Date**: 07/*18*/22

**Step 0**: Mount (connect to) your google drive folder where you want to save the simulation results and model parameters.

In [34]:
from google.colab import drive
drive.mount('/content/drive')
#drive.mount("/content/drive", force_remount=True)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [35]:
# check python version 
from platform import python_version

print(python_version())

3.8.16


In [36]:
# check tensorflow version
import tensorflow as tf
print(tf.__version__)

2.9.2


In [37]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 13.6 gigabytes of available RAM

Not using a high-RAM runtime


**Step 1**: Load module in Google Drive

In [38]:
# The path to save the models and read data from

### !!! please create and name the main folder that you would like to save all results -- ensure that the functions and data files are in this folder
folder = 'Gait_signatures/'


path = '/content/drive/My Drive/' + folder

# Insert the directory
import sys
sys.path.insert(0,path)

**Step 2**: Import necessary packages and functions to develop model

In [39]:
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.python.keras.layers.recurrent import LSTM
from sklearn.model_selection import train_test_split
import sklearn.model_selection as model_selection
import matplotlib.pyplot as plt
import math
import keras as k
import pandas as pd
import numpy as np
from copy import copy
import scipy.io
from sklearn.decomposition import PCA

from scipy.signal import find_peaks
from scipy import interpolate
from scipy.special import iv
from numpy import sin,cos,pi,array,linspace,cumsum,asarray,dot,ones
from pylab import plot, legend, axis, show, randint, randn, std,lstsq

from sklearn.manifold import MDS
import csv
import os
from tqdm import tqdm 

from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt

In [40]:
import fourierseries
import util
import phaser
import dataloader
# Preprocess data for a single subject - to be send to modeling frameworks
def find_phase(k):
    """
    Detrend and compute the phase estimate using Phaser
    INPUT:
      k -- dataframe
    OUTPUT:
      k -- dataframe
    """
    #l = ['hip_flexion_l','hip_flexion_r'] # Phase variables = hip flexion angles
    y = np.array(k)
    print(y.shape)
    y = util.detrend(y.T).T
    print(y.shape)
    phsr = phaser.Phaser(y=y)
    k[:] = phsr.phaserEval(y)[0,:]
    return k

In [41]:
def vonMies(t,t_0, b):
    out = np.exp(b*np.cos(t-t_0))/(2*pi*iv(0, b))
    return out

**Step 3**: Load in data and specify variables/parameters

In [42]:
# Non-changing variables 

# number of trials in dataset 
trialnum = 72 # 72 total trials

# number of samples in each trial
trialsamp = 1500

# number of features collected per trial
feats = 6

#Batch size - same as the number of traintrials
batch_size = trialnum

# Number of Layers
numlayers = 1

# Choose the number of iterations to train the model- if this script has been run previously enter a value greater than was 
# inputted before and rerun the script. 
finalepoch = 10000

# load the input data/kinematics
datafilepath = path + 'PareticvsNonP_RNNData.csv' #input data
all_csvnp = np.loadtxt(datafilepath,delimiter=',').T

# reshape all the input data into a tensor
all_inputdata_s = all_csvnp.reshape(trialnum,trialsamp,feats) 

print('original input data shape is: '+ str(all_csvnp.shape ))
print('input data reshaped is: '+ str(all_inputdata_s.shape))


original input data shape is: (108000, 6)
input data reshaped is: (72, 1500, 6)


**Step 4**: List of model architectures and corresponding variables from script 1 and 2

In [43]:
# generate a list of models and corresponding parameters to test 
test_model_nodes = [512] 
seqs = [249,499] #lookback parameter

# run multiple model architechtures many times to test stability of cost function outputs
runs = 1 # stability analysis - repeat each model architecture this many times
test_model_seq = np.repeat(seqs, runs)

All_nodes = np.empty([0,1], dtype='int')
All_seq = np.empty([0,1],dtype='int')
All_valseg = np.empty([0,1],dtype='int')
All_trainseg = np.empty([0,1],dtype='int')
All_modelname = []
All_mod_name = []

count = 0; #initialize model run -- this serves as the model run ID number
for a in test_model_nodes:
  for b in test_model_seq:
    if count < runs:
      count = count + 1 
    else: 
      count = 1 # reset counter when all runs of certain model attained
    #if statement for valseg, trainseg based on sequence length
    if int(b) == 249:
      trainseg = 4
      valseg = 2
    elif int(b) == 499: 
      trainseg = 2
      valseg = 1
    elif int(b) == 749:
      trainseg = 1
      valseg = 1

    All_nodes = np.append(All_nodes, a) 
    All_seq = np.append(All_seq, int(b))
    All_valseg = np.append(All_valseg, valseg)
    All_trainseg = np.append(All_trainseg, trainseg)
    All_modelname = np.append(All_modelname, 'run_' + str(count) + '_UNIT_' + str(a) + '_LB_' + str(b) + '/' )
    All_mod_name = np.append(All_mod_name, 'run_' + str(count) + '_UNIT_' + str(a) + '_LB_' + str(b) )

**Step 5**: Access and load trained model architectures in loop, extract the external and self-driven Hs and Cs and develop signatures. 

In [44]:
for j in range(len(All_mod_name)):
    # make folder to store model
    newfoldpath = path + All_mod_name[j]

    # extract path to store each model and generated data
    savepath = path + All_modelname[j]
    mod_name = All_mod_name[j]
    modnum = j+1 #model number counter for print statement
    print('Working on: ' + mod_name + ' model ' + str(modnum) + ' / ' + str(len(All_mod_name)))
    
    # Specify variables for model run instance

    # Number of Units
    numunits = All_nodes[j]
    
    test_ind = np.arange(0, len(all_inputdata_s), 1)  # test indices: run through all input data
    ext_drive= trialsamp # length of externally driven prediction

    # PCA of HCs
    ExternalDriveHCs = np.load(savepath + mod_name + '_extdriveHCs.npy')
    SelfDriveHCs = np.load(savepath + mod_name + '_selfdriveHCs.npy')

    X = ExternalDriveHCs # PCA of only externally driven predictions [used to make Gait Signatures]

    pca = PCA(n_components=numunits*2) #initialize

    X_reduction = pca.fit_transform(X) # original data is projected onto the derived principal components from pca

    np.save(savepath + mod_name + '_pca_var.npy', pca.explained_variance_ratio_) #save variance explained by PCs
    #scipy.io.savemat(savepath + mod_name + '_pca_var_all.mat', {'Variance_exp': pca.explained_variance_ratio_}) #save to Matlab

    tot_len = len(test_ind) # total number of trials in dataset
    HC_CellArray = np.empty(shape=[tot_len, ext_drive, X_reduction.shape[1]])
    start = 0
    last = ext_drive # length of trials
    for p in range(tot_len):
        HC_CellArray[p]= X_reduction[start:last]
        start = start+ext_drive
        last = last+ext_drive

    #scipy.io.savemat(savepath + mod_name + '_HCvalues.mat', {'HC_CellArray': HC_CellArray}) save HC time traces to matlab

    # Generate phase averaged signatures
    print('generating phase averaged signatures')
    # phase length = 100
    lim = tot_len
    L = 6 # lets look at 1st 6 to speed things up #numunits*2  # number of PCs or units to be phase averaged
    PhaseAveragedPCs = np.empty(shape=[lim, L, 100])
    PhaseAveragedPCs_shift = np.empty(shape=[lim, L, 100])
    Phase_Variables = np.empty(shape=[lim, 1500])
    cyclephase = []
    PC_Shifts = np.empty(shape=[lim]) # store phase shift values
    # Average orbits initialization
    numSegments = 100 # we want each Phase averaged orbit to be 100 sample points long
    phaseVals = np.linspace(0, 2*pi, numSegments, endpoint=True) # phases that we want our phase averaged orbits to correspond to
    kappa = 20 

    for a in range(lim): #for length of all the trials 
        trialnum = a+1
        print(['processing trial: ' + str(trialnum)]) 
        dats=[] # reset variables after each trial - kinematics
        dats2 = [] # reset the HC params
        allsigs = np.empty(shape=[100,])
        all_phase_var = np.empty(shape=[100,]) 
        PhaseAvgPCs = np.empty(shape=[100])
        all_cyclephase = []
        raw = HC_CellArray[a][:,0:3].T # Only use 1st 3 PCs (of HCs),
        raw2 = HC_CellArray[a][:,0:6].T  # extract all the HC data
        for b in range(6): #Shai's code does this duplication- works better than without duplicating data
            dats.append(raw-raw.mean(axis=1)[:,np.newaxis]) #center the HCs data for phase estimation -- this centers each one separately
            dats2.append(raw2-raw2.mean(axis=1)[:,np.newaxis])
        #rHC = dats2[0] #centered data
        rHC = raw2 #uncentered data
        phr = phaser.Phaser(dats) # use centered data from 1st 3 PCs
        phi = [ phr.phaserEval( d ) for d in dats ] # extract phase
        phi2  =(phi[0].T % (2*pi)); # Take modulo s.t. phase ranges from [0,2*pi]
    
        # find average orbits
        avgOrbits = np.zeros((numSegments,L)) #initialize avg orbits
        phases = np.reshape(phi2,[1500,]) 

        for c in range(numSegments): #number of points in final average orbit/phase averaging
            vonMiesCurrent = vonMies(phases,phaseVals[c],kappa) # for each value of phase points calculate current
            sumVal = np.sum(vonMiesCurrent)

            for d in range(L):  # for the number of features or units
                data = np.reshape(rHC[d,:],[trialsamp,1])
                avgOrbits[c,d] = np.sum(data.T*vonMiesCurrent)/sumVal # phase point (row), feature of PC (column) -- this is generating a value for the 1st phase point of each feature

        PhaseAveragedPCs[a] = avgOrbits.T # transform to match overall saving structure as before
        phi2 = phi2.reshape(1500,)
        Phase_Variables[a] = phi2 # store phase variables for all cycles/features - may need to phase shift these 
        
        # Phase shift the data according to PC1 max align with zero phase
        PC1_maxloc = np.argmax(PhaseAveragedPCs[a][0]) # identify the max PC1 value (1st max if repeated)
        Data2Shift = PhaseAveragedPCs[a] 
        NewP = np.roll(Data2Shift, -PC1_maxloc,axis=1) #shift data so that PC1 max is at the orgin - this is defined as zero phase
        PC_Shifts[a] = PC1_maxloc
        PhaseAveragedPCs_shift[a] = NewP #store shifted data


    np.save(savepath + mod_name + '_PhaseAvgPcs.npy', PhaseAveragedPCs) #unshifted
    scipy.io.savemat(savepath + mod_name + '_PhaseAvgPcs.mat', {'PhaseAvgPCs': PhaseAveragedPCs})

    np.save(savepath + mod_name + '_PhaseAvgPcs_shift.npy', PhaseAveragedPCs_shift) #shifted to set phase = 0 as the max of PC1
    scipy.io.savemat(savepath + mod_name + '_PhaseAvgPcs_shift.mat', {'PhaseAvgPCs_shift': PhaseAveragedPCs_shift})

    np.save(savepath + mod_name + '_PhaseVariables.npy', Phase_Variables)
    scipy.io.savemat(savepath + mod_name + '_PhaseVariables.mat', {'PhaseVariables': Phase_Variables})


    # Concatenate gait signature per trial
    # Make a single phase averaged signature per row
    PA_shape = PhaseAveragedPCs_shift.shape
    gait_sig_size = PA_shape[1]*PA_shape[2]
    Gaitsignatures = np.empty(shape = [len(PhaseAveragedPCs_shift),gait_sig_size]) # Store gait signatures
    Gaitsignatures_trunc6 = np.empty(shape = [len(PhaseAveragedPCs_shift),600]) # Store truncated version of gait signatures to the first 6 PCs
    for h in range(len(PhaseAveragedPCs_shift)):
        reshape_sig = PhaseAveragedPCs_shift[h].reshape(1,gait_sig_size)
        Gaitsignatures[h] = reshape_sig[0]
        Gaitsignatures_trunc6[h] = reshape_sig[0][0:600] #truncate to 1st 6 PCs
    np.save(savepath + mod_name + '_Gaitsignatures.npy', Gaitsignatures)
    scipy.io.savemat(savepath + mod_name + '_Gaitsignatures.mat', {'GaitSigs': Gaitsignatures})

    np.save(savepath + mod_name + '_Gaitsignatures_trunc6.npy', Gaitsignatures_trunc6)
    scipy.io.savemat(savepath + mod_name + '_Gaitsignatures_trunc6.mat', {'GaitSigs': Gaitsignatures_trunc6})

    # Import list of trials' corresponding speed labels
    datafilepath3 = path + 'SpeedLabels.mat'
    speedlabels = scipy.io.loadmat(datafilepath3)
    speedlist = []
    for m in test_ind:
        speedlist.append(speedlabels['SpeedTrainTrials'][m][0][0][0])

    # Import Subject Labels
    myfile = path + 'Subjectlabels.csv'
    csvfile = open(myfile, 'r')
    reader = csv.reader(csvfile, delimiter='\t')
    allsubs = list(reader)
    select_subs = map(allsubs.__getitem__, test_ind)
    subs= list(select_subs)

    # Mutlidimensional scaling of gait signatures
    embedding = MDS(n_components=2)
    X_transformed = embedding.fit_transform(Gaitsignatures_trunc6) # Perform MDS on 6D gait signatures to visualize in 2D
    np.save(savepath + mod_name + '_MDS_X_transformed.npy', X_transformed)

    # MDS plots of ext and self sigs
    fig = plt.figure()
    plt.scatter(X_transformed[:,0],X_transformed[:,1],c = speedlist) # color 2D gait signatures according to speed
    # Loop for annotation of all points
    count = 0
    for i in range(len(subs)):
        plt.annotate(str(count), (X_transformed[i,0], X_transformed[i,1] + 0.3)) # annotate subject ID next to the 2D gait signatures
        count = count + 1    
    plt.savefig(savepath + mod_name +  '_MDS.png', dpi = 300)
    #plt.close(fig)# close figure in loop
    plt.style.use('seaborn-poster')
    
    %matplotlib notebook
    
    # Plot Able-bodied Gait Signatures (3D loops)
    fig = plt.figure(figsize = (8,8))
    ax = plt.axes(projection='3d')
    ax.grid()
    for k in range(30):
      PC1 = Gaitsignatures_trunc6[k,0:100]
      PC2 = Gaitsignatures_trunc6[k,100:200]
      PC3 = Gaitsignatures_trunc6[k,200:300]
      ax.plot3D(PC1, PC2, PC3,'b')
    ax.set_title('Able-bodied gait signatures')

    # Set axes label
    ax.set_xlabel('PC 1', labelpad=20)
    ax.set_ylabel('PC 2', labelpad=20)
    ax.set_zlabel('PC 3', labelpad=20)

    plt.savefig(savepath + mod_name +  '_GaitSignatures_AbleBodied.png', dpi = 300)
    plt.show()

    # Plot Stroke Gait Signatures (3D loops)
    fig = plt.figure(figsize = (8,8))
    ax = plt.axes(projection='3d')
    ax.grid()
    for k in range(42):
      PC1 = Gaitsignatures_trunc6[k+30,0:100]
      PC2 = Gaitsignatures_trunc6[k+30,100:200]
      PC3 = Gaitsignatures_trunc6[k+30,200:300]
      ax.plot3D(PC1, PC2, PC3,'r')
    ax.set_title('Stroke gait signatures')

    # Set axes label
    ax.set_xlabel('PC 1', labelpad=20)
    ax.set_ylabel('PC 2', labelpad=20)
    ax.set_zlabel('PC 3', labelpad=20)

    plt.savefig(savepath + mod_name +  '_GaitSignatures_Stroke.png', dpi = 300)
    plt.show()






Working on: run_1_UNIT_512_LB_249 model 1 / 2
generating phase averaged signatures
['processing trial: 1']
['processing trial: 2']
['processing trial: 3']
['processing trial: 4']
['processing trial: 5']
['processing trial: 6']
['processing trial: 7']


/content/drive/My Drive/Gait_signatures/phaser.py:399: UserWarning: PhaserForSample:nCycMismatch
  warnings.warn( "PhaserForSample:nCycMismatch" )


['processing trial: 8']
['processing trial: 9']
['processing trial: 10']
['processing trial: 11']
['processing trial: 12']
['processing trial: 13']
['processing trial: 14']
['processing trial: 15']
['processing trial: 16']
['processing trial: 17']
['processing trial: 18']
['processing trial: 19']
['processing trial: 20']
['processing trial: 21']
['processing trial: 22']
['processing trial: 23']
['processing trial: 24']
['processing trial: 25']
['processing trial: 26']
['processing trial: 27']
['processing trial: 28']
['processing trial: 29']
['processing trial: 30']
['processing trial: 31']
['processing trial: 32']
['processing trial: 33']
['processing trial: 34']
['processing trial: 35']
['processing trial: 36']
['processing trial: 37']
['processing trial: 38']
['processing trial: 39']
['processing trial: 40']
['processing trial: 41']
['processing trial: 42']
['processing trial: 43']


/content/drive/My Drive/Gait_signatures/phaser.py:397: UserWarning: PhaserForSample:tooShort
  warnings.warn( "PhaserForSample:tooShort" )


['processing trial: 44']
['processing trial: 45']
['processing trial: 46']
['processing trial: 47']
['processing trial: 48']
['processing trial: 49']
['processing trial: 50']
['processing trial: 51']
['processing trial: 52']
['processing trial: 53']
['processing trial: 54']
['processing trial: 55']
['processing trial: 56']
['processing trial: 57']
['processing trial: 58']
['processing trial: 59']
['processing trial: 60']
['processing trial: 61']
['processing trial: 62']
['processing trial: 63']
['processing trial: 64']
['processing trial: 65']
['processing trial: 66']
['processing trial: 67']
['processing trial: 68']
['processing trial: 69']
['processing trial: 70']
['processing trial: 71']
['processing trial: 72']


...

/content/drive/My Drive/Gait_signatures/phaser.py:397: UserWarning: PhaserForSample:tooShort
  warnings.warn( "PhaserForSample:tooShort" )


['processing trial: 44']
['processing trial: 45']
['processing trial: 46']
['processing trial: 47']
['processing trial: 48']
['processing trial: 49']
['processing trial: 50']
['processing trial: 51']
['processing trial: 52']
['processing trial: 53']
['processing trial: 54']
['processing trial: 55']
['processing trial: 56']
['processing trial: 57']
['processing trial: 58']
['processing trial: 59']
['processing trial: 60']
['processing trial: 61']
['processing trial: 62']
['processing trial: 63']
['processing trial: 64']
['processing trial: 65']
['processing trial: 66']
['processing trial: 67']
['processing trial: 68']
['processing trial: 69']
['processing trial: 70']
['processing trial: 71']
['processing trial: 72']


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>